In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
from unidecode import unidecode
import unicodedata
import numpy as np
import math as m
import re

#------------ Remueve acentos 
def remove_accents(input_str):
    # Normalizar el string a su forma combinada
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    # Filtrar y mantener solo los caracteres que no son diacríticos
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
# ----------- Crea data de zonas 

def Transformación_MW_MVAR(cadena):
    #cadena_sin_mas = cadena[1:]
    #final= float(cadena_sin_mas)/(10**6)
    final= float(cadena)/(10**6)
    final=round(final,2)
    return (final)
def obtener_V_op(cadena):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', cadena)
    indices_pares = [i for i in range(len(numeros)) if i % 2 == 0]
    valores_pares = [numeros[i] for i in indices_pares]
    lista_V = [round(float(elemento),2) for elemento in valores_pares]
    if lista_V[0] == lista_V[1] == lista_V[2]:
        return lista_V[0]
    else:
        return "desbanceado" 

In [2]:
# FUNCIONES IMPORTANTES
 
# toma un el string que tiene "V+anglo y me entrega solo el "V y redondeado" 
def separar_numeros(s):
    numeros = re.findall(r'[+-]?\d+\.\d+E[+-]?\d+', s)
    if numeros:
        numero_1 = float(numeros[0])  # Convertir a float
        return round(numero_1, 2)
# obtengo cual es la tension nominal a partir del resultado
def analizar_numero(n):
    if 90 <= n + 20 <= 130 or 90 <= n - 20 <= 130:
        return 110
    elif 200 <= n + 20 <= 240 or 200 <= n - 20 <= 240:
        return 220
    elif 46 <= n + 20 <= 86 or 46 <= n - 20 <= 86:
        return 66
    elif 3 <= n + 10 <= 23.2 or 3 <= n - 10 <= 23.2:
        return 13.2    
    else:
        return None  # Si no cae en ninguna categoría


# Resultados: LOAD FLOW (PV,WP,SG)

In [3]:
# Aqui coloco el nombre del archivo como string
#archivo =  "SEN 2025 VRE Peak 04072024 DBC 85 GFM New - H1_v4_lf.html"
archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"
excel = "Datos_nom_v1.xlsx"

def Resultados_Load_Flow(archivo_html,excel):

    Datos_Nominal_extras = pd.read_excel(excel)
    df= pd.read_html(archivo_html)
    columnas = ['Tensión en Bornes [kV]','Potencia Activa [MW]', 'Potencia Reactiva [Mvar]']
    columnas2 = ['Name1','Name2', 'EMTP Load Flow Component']
    df2 = pd.DataFrame(columns=columnas2)
    
    # Crear el DataFrame vacío con las columnas definidas
    df_FINAL = pd.DataFrame(columns=columnas)
    df=pd.DataFrame(df[0])
    # Se procede a transformar el dataframe de html para que este legible 

    #obtengo el nombre las columnas y las reescribo
    fila_cero = df.iloc[0] 
    df.columns =fila_cero.to_list()
    #borro la fila 0 que tenia los nombres de las columnas
    df = df.iloc[1:]
    # reseteo los indices desde 0 y luego borro la primera columna indice
    df=pd.DataFrame(df.reset_index())
    lista_columnas = fila_cero.to_list()
    #  obtengo solo las 5 primeras importantes
    lista_columnas= lista_columnas[:5] 
    df2 = df[lista_columnas[:5]]
    # Crear una lista que solo contenga las centrales y no las cargas 
 
    valores_filtrar = ['PVbus', 'Slack', 'PQbus']
    df_filtrado = df2[df2['Type'].isin(valores_filtrar)]

    df_FINAL['Potencia Activa [MW]']=df_filtrado['P (W)'].apply(Transformación_MW_MVAR)
    df_FINAL['Potencia Reactiva [Mvar]']=df_filtrado['Q (VAR)'].apply(Transformación_MW_MVAR)
    df_FINAL["Tensión en Bornes [kV]"] = df_filtrado["Vabc (kVRMSLL,deg) phasor"].apply(obtener_V_op)
    df_FINAL['Tensión Nominal [kV]'] = df_FINAL["Tensión en Bornes [kV]"].apply(analizar_numero)
   

    #aqui la colummna device la expando para obtener 3 columnas de tal forma que coincida con lo anterior
    nuevo_df = df_filtrado['Device'].str.split('/', expand=True)
    nuevo_df.columns = ['Name1', 'Name2', 'NameLF',"Name4"]

    # Esto entrega la tabla de resultados de todas las centrales del HTML
    df_Resultados = pd.concat([nuevo_df, df_FINAL], axis=1)
    pd.set_option('display.max_rows', 140)

    #---------------------------- saco los acentos 
    df_Resultados['Name1'] = df_Resultados['Name1'].apply(lambda x: unidecode(x))
    df_Resultados['Name2'] = df_Resultados['Name2'].apply(lambda x: unidecode(x))
    df_Resultados = df_Resultados.drop(columns=['Name4']) #no tiene nada util

    ##------------------------Sobre escribo datos nominales------------------------------------
    
    N = len(df_Resultados)
    for fila in range(N):
        Name1= df_Resultados.at[fila,"Name1"]
        Name2= df_Resultados.at[fila,"Name2"]
        NameLF= df_Resultados.at[fila,"NameLF"]
        result = Datos_Nominal_extras[(Datos_Nominal_extras['Name1'] == Name1) &
                                 (Datos_Nominal_extras['Name2'] == Name2) & 
                                 (Datos_Nominal_extras['NameLF'] == NameLF)]
        if len(result)== 1:
            df_Resultados.at[fila,"Tensión Nominal [kV]"] =result["Tensión Nominal [kV]"]
        else: 
            continue 
    

    df_Resultados['Tensión [pu]'] = df_Resultados["Tensión en Bornes [kV]"]/df_Resultados['Tensión Nominal [kV]']

    return(df_Resultados)

Tabla_LF_GEN=Resultados_Load_Flow(archivo,excel)
Tabla_LF_GEN

C:\Users\fabian.medina\AppData\Local\Temp\ipykernel_19380\1840783760.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_Resultados.at[fila,"Tensión Nominal [kV]"] =result["Tensión Nominal [kV]"]


,Name1,Name2,NameLF,Tensión en Bornes [kV],Potencia Activa [MW],Potencia Reactiva [Mvar],Tensión Nominal [kV],Tensión [pu]
0,Central_Ralco,Ralco_U1,LF_RALC_U1,14.00,96.77,11.43,13.800,1.014493
1,Central_Alfalfal,Alfalfal_U1,LF_ALFAL_U1,12.12,53.46,7.84,12.000,1.010000
2,Central_Alfalfal,Alfalfal_U2,LF_ALFAL_U2,12.12,53.46,8.20,13.800,0.878261
3,Central_Alfalfal_II,Alfalfal_II_U1,LF_ALFAL_II_U1,15.60,39.20,25.80,15.000,1.040000
4,Central_Alfalfal_II,Alfalfal_II_U2,LF_ALFAL_II_U2,15.60,39.20,25.80,15.000,1.040000
5,Central_Angamos,Angamos_U2,LF_ANG_U2,18.00,100.00,-27.44,18.000,1.000000
6,C_ANGOSTURA,Angostura_U1,LF2,13.80,112.30,-5.10,13.800,1.000000
7,Central_Canutillar,Canutillar_U1,LF_CANU_U1,14.20,25.00,5.81,13.800,1.028986
8,Central_Canutillar,Canutillar_U2,LF_CANU_U2,14.20,25.00,5.81,13.800,1.028986
9,Central_Colbun,Colbun_U1,LF_COLB_U1,14.20,100.00,13.84,13.800,1.028986


# Resultados: LOAD FLOW Cargas

In [ ]:
archivo = "SEN 2025 VRE Peak 85 VRE_g1_10 steps_lf.html"


def Resultados_Load_Flow_cargas(archivo_html):


    df= pd.read_html(archivo_html)
    columnas = ['Tensión en Bornes [kV]','Potencia Activa [MW]', 'Potencia Reactiva [Mvar]']
    columnas2 = ['Name1','Name2', 'EMTP Load Flow Component']
    df2 = pd.DataFrame(columns=columnas2)
    
    # Crear el DataFrame vacío con las columnas definidas
    df_FINAL = pd.DataFrame(columns=columnas)
    df=pd.DataFrame(df[0])
    # Se procede a transformar el dataframe de html para que este legible 

    #obtengo el nombre las columnas y las reescribo
    fila_cero = df.iloc[0] 
    df.columns =fila_cero.to_list()
    #borro la fila 0 que tenia los nombres de las columnas
    df = df.iloc[1:]
    # reseteo los indices desde 0 y luego borro la primera columna indice
    df=pd.DataFrame(df.reset_index())
    lista_columnas = fila_cero.to_list()
    #  obtengo solo las 5 primeras importantes
    lista_columnas= lista_columnas[:5] 
    df2 = df[lista_columnas[:5]]
    # Crear una lista que solo contenga las centrales y no las cargas 


    valores_filtrar2 = ['PQload']
    df_filtrado2 = df2[df2['Type'].isin(valores_filtrar2)]
 
    #valores_filtrar = ['PVbus', 'Slack', 'PQbus']
    #df_filtrado = df2[df2['Type'].isin(valores_filtrar)]

    df_FINAL['Potencia Activa [MW]']=df_filtrado2['P (W)'].apply(Transformación_MW_MVAR)
    df_FINAL['Potencia Reactiva [Mvar]']=df_filtrado2['Q (VAR)'].apply(Transformación_MW_MVAR)
    df_FINAL["Tensión en Bornes [kV]"] = df_filtrado2["Vabc (kVRMSLL,deg) phasor"].apply(separar_numeros)
    df_FINAL['Tensión Nominal [kV]'] = df_FINAL["Tensión en Bornes [kV]"].apply(analizar_numero)
    df_FINAL['Tensión [pu]'] = df_FINAL["Tensión en Bornes [kV]"]/df_FINAL['Tensión Nominal [kV]']
  

    nuevo_df2 =df_filtrado2["Device"] 
    # Esto entrega la tabla de resultados de todas las centrales del HTML
    
    df_Resultados2 = pd.concat([nuevo_df2, df_FINAL], axis=1)



    #---------------------------- saco los acentos 
    #df_Resultados['Name1'] = df_Resultados['Name1'].apply(lambda x: unidecode(x))
    #df_Resultados['Name2'] = df_Resultados['Name2'].apply(lambda x: unidecode(x))


    ##------------------------------------------------------------------------------------------
   
    # que me entregue las columnas de 3 en 3
    df_Resultados2 = df_Resultados2.iloc[::3]
    #reseteo los indices 
    df_Resultados2 = df_Resultados2.reset_index(drop=True)
    #borro el nombre "Load_a del string de su"
    df_Resultados2['Device'] = df_Resultados2['Device'].str.replace('/Load_a', '', regex=False)
    return(df_Resultados2)

Tabla_LF_cargas= pd.DataFrame(Resultados_Load_Flow_cargas(archivo))
pd.set_option('display.max_rows', 200)
Tabla_LF_cargas

# Resultados: LOAD FLOW Cargas

In [ ]:
#Resultados_Load_Flow2(archivo)
#a= pd.DataFrame(Resultados_Load_Flow_cargas(archivo))
with pd.ExcelWriter('Resultados_g1.xlsx') as writer:

    Tabla_LF_cargas.to_excel(writer, sheet_name='LFcargas',index=False)
    Tabla_LF_GEN.to_excel(writer, sheet_name='LFGEN',index=False)
